In [82]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import color

In [83]:
def detect_harris_corners(image_path, output_path):
    """
    对输入图像进行 Harris 角点检测，并在角点位置标记红色后保存结果
    """
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray_f = np.float32(gray)
    harris_response = cv2.cornerHarris(gray_f, blockSize=2, ksize=3, k=0.04)
    harris_response = cv2.dilate(harris_response, None)
    img[harris_response > 0.01 * harris_response.max()] = [0, 0, 255]
    cv2.imwrite(output_path, img)

In [84]:
def extract_sift_features(image_path):
    """
    使用 SIFT 提取图像的关键点和描述子
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(img, None)
    return keypoints, descriptors

In [85]:
def extract_hog_features(image_path, patch_size=32):
    """
    利用 Harris 算法检测角点，然后对每个角点周围提取固定大小（patch_size x patch_size）的图像块，
    并计算 HOG 描述子，返回关键点（cv2.KeyPoint 列表）和描述子（numpy 数组）。
    """
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    gray_f = np.float32(img)
    harris_response = cv2.cornerHarris(gray_f, blockSize=2, ksize=3, k=0.04)
    harris_response = cv2.dilate(harris_response, None)
    threshold = 0.01 * harris_response.max()
    # 获取角点坐标 (y, x)
    coords = np.argwhere(harris_response > threshold)
    
    keypoints = []
    descriptors = []
    half = patch_size // 2
    for (y, x) in coords:
        # 确保图像块在图像内
        if x - half < 0 or x + half >= img.shape[1] or y - half < 0 or y + half >= img.shape[0]:
            continue
        patch = img[y-half:y+half, x-half:x+half]
        hog_desc = hog(patch, pixels_per_cell=(8,8), cells_per_block=(2,2), feature_vector=True)
        descriptors.append(hog_desc)
        # 转换 x,y 为 float 类型构造 cv2.KeyPoint 对象
        keypoints.append(cv2.KeyPoint(float(x), float(y), patch_size))
    
    if len(descriptors) == 0:
        descriptors = None
    else:
        descriptors = np.array(descriptors, dtype=np.float32)
        # 如果只有一个角点，则确保 descriptors 为二维数组
        if descriptors.ndim == 1:
            descriptors = descriptors[np.newaxis, :]
    return keypoints, descriptors



In [86]:
def match_features(desc1, desc2):
    """
    使用暴力匹配器对两组描述子进行匹配，并返回按距离排序的匹配结果
    """
    # 确保 descriptor 为连续的 numpy 数组
    desc1 = np.ascontiguousarray(desc1, dtype=np.float32)
    desc2 = np.ascontiguousarray(desc2, dtype=np.float32)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)
    matches = bf.match(desc1, desc2)
    matches = sorted(matches, key=lambda x: x.distance)
    return matches

In [87]:
def extract_and_match_features(img1_path, img2_path, method, match_output_path):
    """
    根据所选描述子类型（SIFT 或 HOG），提取图像的关键点与描述子，然后匹配，
    并将匹配结果绘制后保存到指定路径。
    """
    if method.upper() == "SIFT":
        kp1, desc1 = extract_sift_features(img1_path)
        kp2, desc2 = extract_sift_features(img2_path)
    elif method.upper() == "HOG":
        kp1, desc1 = extract_hog_features(img1_path)
        kp2, desc2 = extract_hog_features(img2_path)
    else:
        raise ValueError("Unsupported method. Please use 'SIFT' or 'HOG'.")
    
    if desc1 is None or desc2 is None or len(kp1) == 0 or len(kp2) == 0:
        raise ValueError("No descriptors found in one of the images.")
    
    matches = match_features(desc1, desc2)
    
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, matches[:50], None, flags=2)
    cv2.imwrite(match_output_path, img_matches)

In [88]:
def stitch_images(img1_path, img2_path, method, output_path):
    """
    根据 method ("SIFT" 或 "HOG") 提取匹配关键点，并利用 RANSAC 求解仿射变换矩阵，
    将第二幅图像变换后与第一幅图像叠加，简单拼接后保存结果。
    """
    if method.upper() == "SIFT":
        kp1, desc1 = extract_sift_features(img1_path)
        kp2, desc2 = extract_sift_features(img2_path)
    elif method.upper() == "HOG":
        kp1, desc1 = extract_hog_features(img1_path)
        kp2, desc2 = extract_hog_features(img2_path)
    else:
        raise ValueError("Unsupported method. Please use 'SIFT' or 'HOG'.")
    
    if desc1 is None or desc2 is None or len(kp1) == 0 or len(kp2) == 0:
        raise ValueError("No descriptors found in one of the images.")
    
    matches = match_features(desc1, desc2)
    if len(matches) < 3:
        raise ValueError("Not enough matches to compute transformation.")
    
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    
    # 使用 RANSAC 计算仿射变换矩阵
    M, mask = cv2.estimateAffinePartial2D(src_pts, dst_pts, method=cv2.RANSAC)
    
    img1 = cv2.imread(img1_path)
    img2 = cv2.imread(img2_path)
    h2, w2 = img2.shape[:2]
    img2_transformed = cv2.warpAffine(img2, M, (w2, h2))
    
    # 简单叠加方式进行拼接（实际应用中可采用更复杂的融合策略）  
    panorama = np.maximum(img1, img2_transformed)
    cv2.imwrite(output_path, panorama)


In [89]:
def stitch_multiple_images(image_paths, output_path):
    """
    基于 SIFT + RANSAC 的方法，依次对多幅图像进行拼接，
    最终将拼接结果保存到 output_path 中。
    """
    result = cv2.imread(image_paths[0])
    result_path = "temp_panorama.png"
    cv2.imwrite(result_path, result)
    for i in range(1, len(image_paths)):
        stitch_images(result_path, image_paths[i], "SIFT", result_path)
        result = cv2.imread(result_path)
    cv2.imwrite(output_path, result)


In [90]:
# 检测角点
detect_harris_corners("images/sudoku.png", "results/sudoku_keypoints.png")

In [91]:
# 提取 SIFT 特征
extract_and_match_features("images/uttower1.jpg", "images/uttower2.jpg", "SIFT", "results/uttower_match_sift.png")

In [92]:
# 提取 HOG 特征
extract_and_match_features("images/uttower1.jpg", "images/uttower2.jpg", "HOG", "results/uttower_match_hog.png")

In [93]:
# 拼接两张图像
stitch_images("images/uttower1.jpg", "images/uttower2.jpg", "SIFT", "results/uttower_stitching_sift.png")
stitch_images("images/uttower1.jpg", "images/uttower2.jpg", "HOG", "results/uttower_stitching_hog.png")

In [94]:
# 拼接多张图像
stitch_multiple_images(["images/yosemite1.jpg", "images/yosemite2.jpg", "images/yosemite3.jpg", "images/yosemite4.jpg"], "results/yosemite_stitching.png")